In [ ]:
import cv2
import numpy as np
from skimage.filters import prewitt, roberts
import matplotlib.pyplot as plt

# Read the image
image_path = 'Grey scale Image_1.png'  # Replace with your image path
image = cv2.imread(image_path)

# Convert BGR to RGB
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2.imwrite('rgb_image.jpg', rgb_image)

# Convert to Grayscale
gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
cv2.imwrite('gray_image.jpg', gray_image)

# Get image size
height, width = gray_image.shape
num_pixels = height * width
print(f"Image Size: {width}x{height}, Total Pixels: {num_pixels}")

# Convert to Binary using a threshold
_, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
cv2.imwrite('binary_image.jpg', binary_image)
black_pixels = np.sum(binary_image == 0)
print(f"Black Pixels Count: {black_pixels}")

# Edge Detection
def edge_detection(image):
    sobel = cv2.Sobel(image, cv2.CV_64F, 1, 1, ksize=3)
    prewitt_edge = prewitt(image)
    roberts_edge = roberts(image)
    canny = cv2.Canny(image, 100, 200)
    return sobel, prewitt_edge, roberts_edge, canny

sobel, prewitt_edge, roberts_edge, canny = edge_detection(gray_image)
cv2.imwrite('sobel_edge.jpg', sobel)
cv2.imwrite('prewitt_edge.jpg', (prewitt_edge * 255).astype(np.uint8))
cv2.imwrite('roberts_edge.jpg', (roberts_edge * 255).astype(np.uint8))

cv2.imwrite('canny_edge.jpg', canny)

# Image Segmentation
_, global_thresh = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
adaptive_thresh = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Watershed Segmentation
def watershed_segmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    unknown = cv2.subtract(sure_bg, sure_fg.astype(np.uint8))
    markers = cv2.connectedComponents(sure_fg.astype(np.uint8))[1]
    markers = markers + 1
    markers[unknown == 255] = 0
    segmented_image = cv2.watershed(image, markers)
    image[segmented_image == -1] = [255, 0, 0]
    return image

watershed_image = watershed_segmentation(image)
cv2.imwrite('watershed_segmented.jpg', watershed_image)

# Save segmentation results
cv2.imwrite('global_threshold.jpg', global_thresh)
cv2.imwrite('adaptive_threshold.jpg', adaptive_thresh)

print("Processing Complete. Images saved successfully.")


Image Size: 512x512, Total Pixels: 262144
Black Pixels Count: 152198
Processing Complete. Images saved successfully.


Black Pixels Count: 262144


Grayscale Image:
